In [ ]:
# Introdution
Python 의 Model I/O에 대해 학습

# FewShotPromptTemplate

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

chat = ChatOpenAI(
    model = "gpt-4.1-nano",
    temperature = 0.1,
    streaming = True,
    callbacks = [
        StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [5]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

chain = prompt | chat

chain.invoke({"country" : "Spain"})

I know this:
Capital: Madrid  
Language: Spanish  
Food: Paella and Tapas  
Currency: Euro

AIMessage(content='I know this:\nCapital: Madrid  \nLanguage: Spanish  \nFood: Paella and Tapas  \nCurrency: Euro', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--7c0c04af-6df5-4f95-98dc-e4ab57b394b5-0')

In [6]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import Runnable
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# 1. Define few-shot examples
examples = [
    {"question": "What do you know about France?", "answer": "It is known for the Eiffel Tower."},
    {"question": "What do you know about Japan?", "answer": "It is famous for Mount Fuji and sushi."}
]

# 2. Define example format
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

# 3. Define few-shot prompt template
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

# 4. Define chat model
chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

# 5. Combine prompt and chat in a chain
chain = prompt | chat

# 6. Run the chain
chain.invoke({"country": "Spain"})

AI: Spain is known for its rich cultural heritage, including flamenco music and dance, delicious cuisine such as paella and tapas, and historical landmarks like the Alhambra and Sagrada Familia. It is also famous for its vibrant festivals, such as La Tomatina and Running of the Bulls. Additionally, Spain has diverse regions, each with its own unique traditions and languages, such as Catalonia and the Basque Country.

AIMessage(content='AI: Spain is known for its rich cultural heritage, including flamenco music and dance, delicious cuisine such as paella and tapas, and historical landmarks like the Alhambra and Sagrada Familia. It is also famous for its vibrant festivals, such as La Tomatina and Running of the Bulls. Additionally, Spain has diverse regions, each with its own unique traditions and languages, such as Catalonia and the Basque Country.', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--09894610-cbe2-49ac-9695-72fd6bd25224-0')

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, ChatMessagePromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    model="gpt-4o-mini",         # Specifies the OpenAI model
    temperature=0.1,             # Low randomness; more deterministic responses
    streaming=True,              # Enables streaming responses token-by-token
    callbacks=[
        StreamingStdOutCallbackHandler()  # Streams output to standard output
    ],
)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [8]:
example_prompt: ChatPromptTemplate = ChatPromptTemplate.from_messages(
    [
        ("human", "what do you know about {country}?"),
        ("ai", "{answer}")
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt: ChatPromptTemplate = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a geography expert, you give short answers."),
        example_prompt,
        ("human", "what do you know abiut {country}")
    ]
)

In [9]:
chain: any= final_prompt | chat
chain.invoke(
    {
        "country", "korea"
    }
)

Korea refers to two countries: 

1. **South Korea** (Republic of Korea)
   - Capital: Seoul
   - Language: Korean
   - Currency: South Korean Won (KRW)

2. **North Korea** (Democratic People's Republic of Korea)
   - Capital: Pyongyang
   - Language: Korean
   - Currency: North Korean Won (KPW)

Both countries share a common cultural and historical background but have distinct political systems.

AIMessage(content="Korea refers to two countries: \n\n1. **South Korea** (Republic of Korea)\n   - Capital: Seoul\n   - Language: Korean\n   - Currency: South Korean Won (KRW)\n\n2. **North Korea** (Democratic People's Republic of Korea)\n   - Capital: Pyongyang\n   - Language: Korean\n   - Currency: North Korean Won (KPW)\n\nBoth countries share a common cultural and historical background but have distinct political systems.", additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--e0a7b394-6e2f-4a44-bf75-44264dbd3db2-0')

In [1]:
#lengthBasedExampleSelector

from langchain_openai import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [2]:
example_prompt: PromptTemplate = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=80
)

In [3]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Brazil")

'Human: What do you know about France?\nAI: \n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

In [4]:
chain = prompt | chat

chain.invoke({"country": "Spain"} )

AI: Here is what I know about Spain:

- **Capital**: Madrid
- **Language**: Spanish (Castilian)
- **Food**: Tapas, Paella, Jamón Ibérico
- **Currency**: Euro
- **Culture**: Known for its rich history, art, flamenco music and dance, and festivals like La Tomatina and Running of the Bulls.
- **Geography**: Located in Southwestern Europe, bordered by France, Portugal, and the Mediterranean Sea. 

If you have specific aspects of Spain you'd like to know more about, feel free to ask!

AIMessage(content="AI: Here is what I know about Spain:\n\n- **Capital**: Madrid\n- **Language**: Spanish (Castilian)\n- **Food**: Tapas, Paella, Jamón Ibérico\n- **Currency**: Euro\n- **Culture**: Known for its rich history, art, flamenco music and dance, and festivals like La Tomatina and Running of the Bulls.\n- **Geography**: Located in Southwestern Europe, bordered by France, Portugal, and the Mediterranean Sea. \n\nIf you have specific aspects of Spain you'd like to know more about, feel free to ask!", additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'service_tier': 'default'}, id='run--5abbb55a-6a8e-4691-82e5-b75c2f2d6882-0')

In [10]:
RandomBasedExampleSelector

from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    model = "gpt-4.1-nano",
    temperature = 0.1,
    streaming = True,
    callbacks = [
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [11]:
class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples: Any = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


In [12]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = RandomExampleSelector(
    examples=examples
)

In [13]:
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about Italy?\nAI: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

In [ ]:
#Serialization_and_Composition

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate
from typing import Any


chat = ChatOpenAI(
    model = "gpt-4.1-nano",
    temperature = 0.1,
    streaming = True,
    callbacks = [
        StreamingStdOutCallbackHandler()
    ]
)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_1720\1506992078.py:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(


In [15]:
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonation a {character}
    """
)

example = PromptTemplate.from_template(
    """
    this is an example of how you talk:

    Huamn: {example_question}
    You: {example_answer}
    """
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
    """
)

final : PromptTemplate = PromptTemplate.from_template(
    """

    {intro}

    {example}

    {start}

    """
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts
)

chain: Any = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_1720\1870182591.py:44: LangChainDeprecationWarning: This class is deprecated in favor of chaining individual prompts together.
  full_prompt = PipelinePromptTemplate(


Arrr! That be a fine question, matey! I be lovin' a hearty feast of salted fish and grog, aye! Nothing beats a good ol' pirate's meal after a day of plunderin' the high seas! Yarrr!

AIMessage(content="Arrr! That be a fine question, matey! I be lovin' a hearty feast of salted fish and grog, aye! Nothing beats a good ol' pirate's meal after a day of plunderin' the high seas! Yarrr!", additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--41c8cccc-a155-41c2-aae7-8e5aec2d8998-0')

In [ ]:


from langchain_openai import ChatOpenAI
from langchain.callbacks import StreamlitCallbackHandler
from langchain.prompts import load_prompt

chat = ChatOpenAI(
    model = "gpt-4o-mini",
    temperature = 0.1,
    streaming = True,
    callbacks = [
        StreamingStdOutCallbackHandler()
    ]
)

prompt = load_prompt("./prompt.json")

prompt.format(country="Germmany")

'What is the capital of Germmany'

In [ ]:
chain = prompt | chat

chain.invoke(
    {
        "country" : "Germmany"
    }
)

The capital of Germany is Berlin.

AIMessage(content='The capital of Germany is Berlin.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'service_tier': 'default'}, id='run--f1bd0873-11a1-4356-8903-631f6f557f85-0')

In [19]:
#Caching

from langchain_openai import ChatOpenAI
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)

chat = ChatOpenAI(
    model = "gpt-4o-mini",
    temperature = 0.1
)

chat.predict("what is the captal of Japan")

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: what is the captal of Japan"
  ]
}
[llm/end] [llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The capital of Japan is Tokyo.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "The capital of Japan is Tokyo.",
            "additional_kwargs": {
              "refusal": null
            },
            "response_metadata": {
              "token_usage": {
                "completion_tokens": 7,
                "prompt_tokens": 14,
                "total_tokens": 21,
                "completion_tokens_details": {
              

'The capital of Japan is Tokyo.'

In [20]:
chat.predict("what is the captal of Korea")

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: what is the captal of Korea"
  ]
}
[llm/end] [llm:ChatOpenAI] [1.38s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The capital of South Korea is Seoul. North Korea's capital is Pyongyang.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "The capital of South Korea is Seoul. North Korea's capital is Pyongyang.",
            "additional_kwargs": {
              "refusal": null
            },
            "response_metadata": {
              "token_usage": {
                "completion_tokens": 17,
                "prompt_tokens": 14,
       

"The capital of South Korea is Seoul. North Korea's capital is Pyongyang."

In [22]:
#serialization
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.globals import set_debug
set_debug(True)

chat : Any = ChatOpenAI(
    model = "gpt-4o-mini",
    temperature = 0.1
)

with get_openai_callback() as usage:
    a: Any = chat.predict("what is the recipt for soju")
    b: Any = chat.predict("what is the recipt for bread")

print(usage)
print(usage.total_cost)        # 전체 비용
print(usage.total_tokens)      # 토큰 총합
print(usage.prompt_tokens)     # 프롬프트에 사용된 토큰수
print(usage.completion_tokens) # 모델이 쓴 토큰

Error in OpenAICallbackHandler.on_llm_end callback: KeyError('total_tokens')
Error in OpenAICallbackHandler.on_llm_end callback: KeyError('total_tokens')


[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: what is the recipt for soju"
  ]
}
[llm/end] [llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Soju is a traditional Korean distilled spirit, typically made from grains or starches such as rice, barley, or sweet potatoes. While you can buy soju at stores, making it at home is a complex process that requires fermentation and distillation. Here’s a simplified overview of how soju is traditionally made:\n\n### Ingredients:\n1. **Base Ingredient**: 2-3 cups of rice, barley, or sweet potatoes\n2. **Water**: Enough to cook the base ingredient and for fermentation\n3. **Nuruk**: A traditional Korean fermentation starter (can be found in Asian markets or online)\n4. **Yeast**: Optional, depending on the fermentation process\n\n### Equipment:\n- Large pot for cooking\n- Fermentation container (glass or food-grade plastic)\n- Distillation apparatus (if you wan

In [24]:
#memory
#ConversationBufferMemory

from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input" : "Hi"}, {"outpu" : "howarre you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='howarre you?', additional_kwargs={}, response_metadata={})]}

In [26]:
#ConversationBufferWindowMemory

from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=5
)

def add_message(input, output):
    memory.save_context({"input" : input}, {"output" : output})

add_message("1","1")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_1720\3339732550.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


In [27]:
for i in range(5):
    add_message(str(1), str(1))

memory.load_memory_variables({})

{'history': [HumanMessage(content='1', additional_kwargs={}, response_metadata={}),
  AIMessage(content='1', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='1', additional_kwargs={}, response_metadata={}),
  AIMessage(content='1', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='1', additional_kwargs={}, response_metadata={}),
  AIMessage(content='1', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='1', additional_kwargs={}, response_metadata={}),
  AIMessage(content='1', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='1', additional_kwargs={}, response_metadata={}),
  AIMessage(content='1', additional_kwargs={}, response_metadata={})]}

In [30]:
#ConversationSummaryMemory

from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"input" : input}, {"output" : output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm KwanTae I live in South Korea", "Wow that is so cool")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_1720\134659016.py:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm)


[chain/start] [chain:LLMChain] Entering Chain run with input:
{
  "summary": "",
  "new_lines": "Human: Hi I'm KwanTae I live in South Korea\nAI: Wow that is so cool"
}
[llm/start] [chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.\n\nEXAMPLE\nCurrent summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.\n\nNew lines of conversation:\nHuman: Why do you think artificial intelligence is a force for good?\nAI: Because artificial intelligence will help humans reach their full potential.\n\nNew summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.\nEND OF EXAMPLE\n\nCurrent summary:\n\n\nNew lines of conversation:\nHuman: Hi I'm 

In [32]:

get_history()

{'history': 'The human introduces themselves as KwanTae from South Korea. The AI responds by expressing admiration for their location.'}

In [35]:
!uv add networkx

Resolved 88 packages in 432ms
Prepared 1 package in 602ms
Installed 1 package in 377ms
 + networkx==3.5


In [36]:

from langchain.memory import ConversationKGMemory
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True
)

async def add_message(input, output):
    await memory.save_context({"input": input}, {"output": output})

async def main():
    await add_message("Hi I'm KwanTae I live in South Korea", "Wow that is so cool")
    history = await memory.load_memory_variables({})
    print(history)


In [37]:
memory.load_memory_variables({"input" : "who is KwanTae"})

[chain/start] [chain:LLMChain] Entering Chain run with input:
{
  "history": "",
  "input": "who is KwanTae"
}
[llm/start] [chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an AI assistant reading the transcript of a conversation between an AI and a human. Extract all of the proper nouns from the last line of conversation. As a guideline, a proper noun is generally capitalized. You should definitely extract all names and places.\n\nThe conversation history is provided just in case of a coreference (e.g. \"What do you know about him\" where \"him\" is defined in a previous line) -- ignore items mentioned there that are not in the last line.\n\nReturn the output as a single comma-separated list, or NONE if there is nothing of note to return (e.g. the user is just issuing a greeting or having a simple conversation).\n\nEXAMPLE\nConversation history:\nPerson #1: how's it going today?\nAI: \"It's going great! How about you?\"\nPerson #1: goo

{'history': []}

In [6]:
#Memory on LLM Chain

#LLM = off the shlf : General Perpose chain (일반적인 목적을 가지는 체인)
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
    memory_key="chat_history"
)

template = """
you are a helpful AI taling a haman.

{chat_history}
Human: {question}
Your:
"""

chain: Any = LLMChain(
    llm=llm,
    memory=memory,
    # prompt=PromptTemplate.from_template("{question}")
    prompt=PromptTemplate.from_template(template=template)
)

chain.predict(question="I live in seoul")

"That's great! Seoul is a vibrant city with a rich history and a mix of modern and traditional culture. What do you enjoy most about living there?"

In [7]:
chain.predict(question="where's do I live")

"AI: You live in Seoul, which is the capital city of South Korea. It's known for its bustling neighborhoods, delicious food, and historical sites. Do you have a favorite area or landmark in the city?"

In [8]:
memory.load_memory_variables({})

{'chat_history': "System: The human mentions living in Seoul, and the AI responds positively, highlighting Seoul's vibrant culture and history, and asks what the human enjoys most about living there.\nHuman: where's do I live\nAI: AI: You live in Seoul, which is the capital city of South Korea. It's known for its bustling neighborhoods, delicious food, and historical sites. Do you have a favorite area or landmark in the city?"}

In [13]:
#ChatBasedMemory
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder

llm = ChatOpenAI(
    model = "gpt-4o-mini",
    temperature = 0.1
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt: Any = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

chain: Any = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True
)

chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: I live in Seoul

> Finished chain.


"That's great! Seoul is a vibrant city with a rich history and a mix of modern and traditional culture. What do you enjoy most about living there?"

In [14]:
chain.predict(question="where's i live")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: I live in Seoul
AI: That's great! Seoul is a vibrant city with a rich history and a mix of modern and traditional culture. What do you enjoy most about living there?
Human: where's i live

> Finished chain.


"If you're looking for specific information about your location in Seoul, I can't access personal data or know your exact address. However, I can help you with general information about neighborhoods, attractions, or anything else related to Seoul. Just let me know what you need!"

In [40]:
#LCEL Based Memory(LangChain Expression Language)

from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(
    model = "gpt-4o-mini",
    temperature = 0.1
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({
        "question": question
    })
    
    memory.save_context({"input": question}, {"output": result.content})
    print(result)

#chain.invoke(
#    {
#        "chat_history" : memory.load_memory_variables({})["chat_history"],
#        "question" : "My name is ksm"
#    }
#)

In [41]:
invoke_chain("My name is ksm")

content='Nice to meet you, ksm! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 25, 'total_tokens': 40, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CKhn5W77jwHCQercTkVTcjn6155gT', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--73638309-31ff-490b-b8e8-ab2629a671a0-0' usage_metadata={'input_tokens': 25, 'output_tokens': 15, 'total_tokens': 40, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [42]:
memory.load_memory_variables({})["chat_history"]

[HumanMessage(content='My name is ksm', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Nice to meet you, ksm! How can I assist you today?', additional_kwargs={}, response_metadata={})]